In [1]:
%matplotlib inline
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt
from sympy import pprint, init_printing, Symbol, exp, Integral, integrate, oo, gamma, Q, refine, log

## Poisson model comparison

Let's find the necessary terms to calculate the Bayes factor:

$$
\frac{p(M_1|x)}{p(M_2|x)} = \frac{p(M_1)}{p(M_2)} \, \frac{p(x|M_1)}{p(x|M_2)}
$$

* [Prior for Poisson scale parameter](#Prior-for-Poisson-scale-parameter)
* [Model 1 evidence](#Model-1-evidence)
* [Model 2 evidence](#Model-2-evidence)
* [Bayes factor](#Bayes-factor)

### Prior for Poisson scale parameter

We're going to determine the lower and upper bounds on the rates we expect to see, and take a prior that makes the *log* of that uniform; this means that, for bounds $a$ and $b$ on the rate, the probability density is given by:

$$
\frac{1}{x \cdot (ln(b) - ln(a))}
$$



### Model 1 evidence

In model 1 we have 1 parameter, $x$, which is a Poisson rate that explains both observations; the likelihood is the product of the Poisson likelihoods of both observations times the prior, which as explained before is log-constant between lower and upper bounds $a$ and $b$:

$$
p(x|M_1) = \int_a^b p(k_1 \mid x) \cdot p(k_2 \mid x) \cdot p(x) dx
$$

$$
\int_a^b \frac{x ^ {k_1} e ^ {- x}}{k_1!} \cdot \frac{x ^ {k_2} e ^ {- x}}{k_2!} \cdot \frac{1}{x \cdot (ln(b) - ln(a))} dx
$$

$$
\frac{1}{k_1! \cdot k_2! \cdot (ln(b) - ln(a))} \cdot \int_a^b x ^ {k_1 + k_2 - 1} \cdot e ^ {-2x} dx
$$

Thanks to maxima, it seems this can be expressed with incomplete Gamma functions:

$$
\frac{1}{k_1! \cdot k_2! \cdot (ln(b) - ln(a))} \cdot 2 ^ {-k_2 -k_1} \cdot \big(\Gamma(k_2 + k_1, 2 a) - \Gamma(k_2 + k_1, 2 b) \big)
$$

In [2]:

def poisson_density(k, x):
    return (x ** k) * exp(-x) / gamma(k + 1)

def logflat_prior(a, b, x):
    return 1 / (x * (log(b) - log(a)))

# model 1
def make_model1():
    a = Symbol("a")
    b = Symbol("b")
    k1 = Symbol("k1")
    k2 = Symbol("k2")
    x = Symbol("x")
    
    likelihood = poisson_density(k1, x) * poisson_density(k2, x) * logflat_prior(a, b, x)
    
    return integrate(likelihood, (x, a, b))

def naive_model1_evidence(a, b, k1, k2):
    constant = np.log(b) - np.log(a)
    def f(x):
        ll = stats.poisson.logpmf(k1, x)
        ll += stats.poisson.logpmf(k2, x)
        ll -= np.log(x)

        return np.exp(ll)

    return scipy.integrate.quad(f, a, b) / constant

# compare symbolic and numeric computations

print(naive_model1_evidence(0.1, 10, 5, 5))
print(make_model1().subs('a', 0.1).subs('b', 10).subs('k1', 5).subs('k2', 5).evalf())

[5.31716311e-03 5.96152204e-13]
0.00531716310935224


### Model 2 evidence

In model 2 we have 2 parameters for the Poisson rates, $x_1$ and $x_2$, each explaining one observation. Each of these has a log-constant prior between $a$ and $b$:

$$
p(x|M_2) = \int_a^b \int_a^b p(k_1 \mid x_1) \cdot p(k_2 \mid x_2) \cdot p(x_1) \cdot p(x_2) dx_1 dx_2
$$

$$
\int_a^b \int_a^b \frac{x_1 ^ {k_1} e ^ {- x_1}}{k_1!} \cdot \frac{x_2 ^ {k_2} e ^ {- x_2}}{k_2!} \cdot \frac{1}{x_1 \cdot (ln(b) - ln(a))} \cdot \frac{1}{x_2 \cdot (ln(b) - ln(a))} dx_1 dx_2
$$

$$
\left( \int_a^b \frac{x_1 ^ {k_1} e ^ {- x_1}}{k_1!} \cdot \frac{1}{x_1 \cdot (ln(b) - ln(a))} dx_1 \right) \cdot \left( \int_a^b \frac{x_2 ^ {k_2} e ^ {- x_2}}{k_2!}  \cdot \frac{1}{x_2 \cdot (ln(b) - ln(a))} dx_2 \right)
$$

In [3]:
# model 2
def make_model2():
    a = Symbol("a")
    b = Symbol("b")
    k1 = Symbol("k1")
    k2 = Symbol("k2")
    x1 = Symbol("x1")
    x2 = Symbol("x2")
    
    term1 = poisson_density(k1, x1) * logflat_prior(a, b, x1)
    term2 = poisson_density(k2, x2) * logflat_prior(a, b, x2)
    
    return integrate(term1, (x1, a, b)) * integrate(term2, (x2, a, b))

def naive_model2_evidence(a, b, k1, k2):
    constant = np.log(b) - np.log(a)
    
    def f1(x):
        ll = stats.poisson.logpmf(k1, x)
        ll -= np.log(x)
        return np.exp(ll)
    
    def f2(x):
        ll = stats.poisson.logpmf(k2, x)
        ll -= np.log(x)
        return np.exp(ll)
    
    term1 = scipy.integrate.quad(f1, a, b)
    term2 = scipy.integrate.quad(f2, a, b)
    
    res = term1[0] * term2[0] / (constant * constant)

    return res

print(naive_model2_evidence(0.1, 10, 5, 5))
print(make_model2().subs('a', 0.1).subs('b', 10).subs('k1', 5).subs('k2', 5).evalf())


0.0017773826940835433
0.00177738269408354


### Bayes factor

Let's take the base2 log of the Bayes factor:

In [10]:
def bayes_factor(a, b, k1, k2):
    m1 = make_model1()
    m2 = make_model2()
    
    res1 = m1.subs('a', a).subs('b', b).subs('k1', k1).subs('k2', k2)
    res2 = m2.subs('a', a).subs('b', b).subs('k1', k1).subs('k2', k2)
    
    return (log(res1, 2) - log(res2, 2)).evalf()

def naive_bayes_factor(a, b, k1, k2):
    m1_evidence = naive_model1_evidence(a, b, k1, k2)[0]
    m2_evidence = naive_model2_evidence(a, b, k1, k2)
    
    return np.log2(m1_evidence) - np.log2(m2_evidence)

print(bayes_factor(0.1, 1000, 50, 40))
print(naive_bayes_factor(0.1, 1000, 50, 40))
    

3.30724319907297
3.3072431990729765
